In [1]:
import os

import pandas as pd
import numpy as np

from keras import layers
from keras import models
from keras.preprocessing import image
import matplotlib.pyplot as plt
import time

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
samuel_images_validation_dir = 'data/crop_images/samuel_images_validation'
samuel_images_train_dir = 'data/crop_images/samuel_images_train'

In [3]:
model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), activation='relu',
                        input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_1 (Separabl (None, 222, 222, 32)      155       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 109, 109, 64)      2400      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 52, 52, 128)       8896      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 24, 24, 128)       17664     
__________

In [5]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        samuel_images_train_dir,
        target_size=(224, 224),
        batch_size=60,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        samuel_images_validation_dir,
        target_size=(224, 224),
        batch_size=60,
        class_mode='binary')

Found 1421 images belonging to 2 classes.
Found 706 images belonging to 2 classes.


In [7]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=120,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/120
100/100 [==============================] - 798s 8s/step - loss: 0.6931 - acc: 0.5116 - val_loss: 0.6910 - val_acc: 0.5751
Epoch 2/120
100/100 [==============================] - 732s 7s/step - loss: 0.6907 - acc: 0.5454 - val_loss: 0.6879 - val_acc: 0.5659
Epoch 3/120
100/100 [==============================] - 735s 7s/step - loss: 0.6787 - acc: 0.5945 - val_loss: 0.6620 - val_acc: 0.6287
Epoch 4/120
100/100 [==============================] - 563s 6s/step - loss: 0.6653 - acc: 0.5977 - val_loss: 0.6352 - val_acc: 0.6624
Epoch 5/120
100/100 [==============================] - 557s 6s/step - loss: 0.6582 - acc: 0.6142 - val_loss: 0.6240 - val_acc: 0.6593
Epoch 6/120
100/100 [==============================] - 560s 6s/step - loss: 0.6560 - acc: 0.6144 - val_loss: 0.6108 - val_acc: 0.6661
Epoch 7/120
100/100 [==============================] - 542s 5s/step - loss: 0.6514 - acc: 0.6253 - val_loss: 0.6150 - val_acc: 0.6654
Epoch 8/120
100/100 [==============================] - 547s 5s

Epoch 62/120
100/100 [==============================] - 311s 3s/step - loss: 0.5681 - acc: 0.7012 - val_loss: 0.5598 - val_acc: 0.7055
Epoch 63/120
100/100 [==============================] - 331s 3s/step - loss: 0.5671 - acc: 0.7100 - val_loss: 0.5639 - val_acc: 0.7021
Epoch 64/120
100/100 [==============================] - 433s 4s/step - loss: 0.5664 - acc: 0.7046 - val_loss: 0.5550 - val_acc: 0.7075
Epoch 65/120
100/100 [==============================] - 486s 5s/step - loss: 0.5680 - acc: 0.6986 - val_loss: 0.5317 - val_acc: 0.7293
Epoch 66/120
100/100 [==============================] - 484s 5s/step - loss: 0.5671 - acc: 0.6996 - val_loss: 0.5359 - val_acc: 0.7242
Epoch 67/120
100/100 [==============================] - 478s 5s/step - loss: 0.5601 - acc: 0.7160 - val_loss: 0.5481 - val_acc: 0.7123
Epoch 68/120
100/100 [==============================] - 477s 5s/step - loss: 0.5575 - acc: 0.7165 - val_loss: 0.5283 - val_acc: 0.7276
Epoch 69/120
100/100 [==============================] -

StopIteration: [Errno 2] No such file or directory: 'data/crop_images/samuel_images_train/samuel/00000029.jpg'

### Save Model

In [8]:
model_file = 'data/crop_images/samuel_images_model_{}.h5'.format(time.time())
model.save(model_file)

print('saved to {}'.format(model_file))

saved to data/crop_images/samuel_images_model_1533399531.290833.h5


### Plot The ROC

In [9]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

### Validate model

In [10]:
samuel_images_test_dir = 'data/crop_images/samuel_images_test'

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        samuel_images_test_dir,
        target_size=(224, 224),
        batch_size=60,
        class_mode='binary')

Found 302 images belonging to 2 classes.


In [12]:
loss, acc = model.evaluate_generator(test_generator)
print('loss={}, acc={}'.format(loss, acc))

loss=0.8738769861246576, acc=0.5860927160212536


### Display predicted items

In [13]:
def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
predictions = model.predict_generator(test_generator)

In [ ]:
test_image_results = pd.DataFrame({
    'prediction': predictions.flatten(),
    'filename': test_generator.filenames
}) \
.sort_values('prediction', ascending=False) \
.head(n=10)

In [ ]:
test_image_results

In [ ]:
fnames = [os.path.join(samuel_images_test_dir, fname) for fname in test_image_results.filename]

imgs = [image.load_img(fname, target_size=(224,224)) for fname in fnames]
show_images(imgs, 5, test_image_results.prediction)

In [ ]:
# Use Grid Search to use all cores https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
